# Imports

In [196]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [197]:
# Basic Python computing
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import pickle
import pandas as pd
pd.set_option('display.max_columns', 200)

# Multiprocessing and other helpful non-necessities
from functools import partial
import multiprocessing
import time
from multiprocessing.pool import ThreadPool
import threading
from tqdm import notebook
notebook.tqdm.pandas()
notebook.tqdm.get_lock().locks = []

# Euchre functions from src
sys.path.insert(0, '/'.join(os.getcwd().split('/')[:-1]) + '/src')
from card import Card
from board import Board
from player import Player
from rule import Rule
import optimal_strategy as opt
import suboptimal_strategy as subopt

# Data functions from src
from euchre_lib import *
from search_lib import *
import data_formatting as dtf
from Timer import Timer
import store_data

In [39]:
import yaml
CFG_FILE = '/'.join(os.getcwd().split('/')[:-1]) + '/config.yaml'
CONFIG = yaml.load(open(CFG_FILE, 'r'), Loader=yaml.FullLoader)

# Create optimal-vs-basic dataset

In [89]:
#outfolder = CONFIG['big_data_dir'] + 'opt_vs_basic'
outfolder = CONFIG['data_dir'] + 'opt_vs_basic'
if not os.path.exists(outfolder): os.mkdir(outfolder)
os.system(f'rm {outfolder}/*')
p0, p2 = opt.make_optimal_player(0), opt.make_optimal_player(2)
p1, p3 = subopt.make_BASIC_player(1), subopt.make_BASIC_player(3)

In [90]:
# Short wrapper to use multiprocessing to run many 
def performance(id, n_epochs, n_hands, p0=None, p1=None, p2=None, p3=None, use_tqdm=False):
    iterable = range(n_epochs)
    if use_tqdm:
        print(' ', end='', flush=True)
        iterable = notebook.tqdm(iterable, desc='Worker '+str(id))
    
    for i in iterable:
        if not use_tqdm:
            print('Worker %i, epoch %i...' %(id, i))
        board = Board(p0=p0, p1=p1, p2=p2, p3=p3)
        for hand in range(n_hands):
            board.play_hand()
        board.writeout(folder=outfolder, keep_results=False)

In [91]:
n_epochs, n_hands = int(1e1), int(1e4)
AVAILABLE_CPUS = 16-1
n_epochs = max(n_epochs, AVAILABLE_CPUS) # Use all the CPUs available
particular_performance_function = partial(performance, n_epochs=int(n_epochs/AVAILABLE_CPUS), n_hands=n_hands, p0=p0, p1=p1, p2=p2, p3=p3)

In [92]:
%%time
pool = multiprocessing.Pool(AVAILABLE_CPUS)
for worker in range(AVAILABLE_CPUS):
    pool.apply_async(particular_performance_function, args=(worker,))
pool.close()
pool.join()
print('Done!')

Worker 1, epoch 0...Worker 3, epoch 0...Worker 0, epoch 0...Worker 2, epoch 0...
Worker 5, epoch 0...Worker 6, epoch 0...Worker 4, epoch 0...
Worker 7, epoch 0...Worker 9, epoch 0...




Worker 8, epoch 0...Worker 11, epoch 0...Worker 12, epoch 0...Worker 10, epoch 0...


Worker 13, epoch 0...


Worker 14, epoch 0...

Done!
CPU times: user 15.8 ms, sys: 40.4 ms, total: 56.2 ms
Wall time: 151 ms


# Synthesize performance

In [66]:
CORRECT_COMMA_COUNT = 124
def clean_csv(folder):
    print('Cleaning folder:', folder)
    for file in os.listdir(folder):
        if '.csv' not in file: continue
        with open(os.path.join(folder, file), 'r') as f:
            lines = f.read().split('\n')
        good_lines = [l for l in lines if len(l.split(',')) == CORRECT_COMMA_COUNT]
        if len(good_lines) < 0.5*len(lines):
            print('BIG ERROR')
            continue
        if len(good_lines)+1 < len(lines):
            print('%s has %i bad lines' %(file, len(lines)-len(good_lines)))
            with open(os.path.join(folder, file), 'w') as f:
                f.write('\n'.join(good_lines))

In [67]:
clean_csv(outfolder)

Cleaning folder: /home/taylora_mitre/euchre-ML/data/opt_vs_basic


In [31]:
clean_csv('/staging/fast/taylora/euchre/opt_vs_inter')

In [32]:
clean_csv('/staging/fast/taylora/euchre/inter_vs_basic')

In [49]:
def get_performance(folder, p02_level, p13_level):
    print('Loading files into memory...')
    if folder[-1] != '/': folder += '/'
    df = pd.concat([pd.read_csv(folder+file) for file in os.listdir(folder) if '.csv' in file]).reset_index(drop=True)
    if len(p02_level) < len(p13_level): p02_level += ' '*(len(p13_level)-len(p02_level))
    else:                               p13_level += ' '*(len(p02_level)-len(p13_level))
    
    print('Applying caller trueid / points...')
    df['caller_trueid'] = df.apply(lambda row: row['p' + str(row['caller']) + 'trueid'], axis=1)
    df['caller_points'] = df.apply(lambda x: 4*(x['result']=='Loner') + 2*(x['result']=='Sweep') +\
                                             1*(x['result']=='Single') - 2*(x['result']=='EUCHRE'), axis=1)
    calls = [df[df['caller_trueid']==i] for i in range(4)]
    
    """
    What do I want to return? What useful statistics can I show to prove that Optimal > Intermediate > Basic?
    
    1.) Avg points per EVERY hand
    2.) Avg points per call
    3.) Rate of euchres / singles / sweeps per call
    4.) Rate of loner successes
    """
    mask_02 = (df['caller_trueid'] == 0) | (df['caller_trueid'] == 2)
    mask_13 = (df['caller_trueid'] == 1) | (df['caller_trueid'] == 3)
    callcount_02 = len(calls[0]) + len(calls[2])
    callcount_13 = len(calls[1]) + len(calls[3])
    
    # Statistic 1: Avg points (by team) per every hand
    points_02 = sum(df[(df['caller_points'] > 0) & (mask_02)]['caller_points']) + sum(df[(df['caller_points'] < 0) & (mask_13)]['caller_points'])
    points_13 = sum(df[(df['caller_points'] > 0) & (mask_13)]['caller_points']) + sum(df[(df['caller_points'] < 0) & (mask_02)]['caller_points'])
    print('Non-negative PPH Team 0/2 (%s): %.2f' %(p02_level, points_02/len(df)))
    print('Non-negative PPH Team 1/3 (%s): %.2f' %(p13_level, points_13/len(df)))
    print()
    
    # Statistic 2: Avg points (by team) per call
    print('PPC Team 0/2 (%s): %.2f' %(p02_level, (sum(calls[0]['caller_points'])+sum(calls[2]['caller_points'])/callcount_02)))
    print('PPC Team 1/3 (%s): %.2f' %(p13_level, (sum(calls[1]['caller_points'])+sum(calls[3]['caller_points'])/callcount_13)))
    print()
    
    # Statistic 3: Rate of euchres / singles / sweeps (by team) per call
    print('Euchre / single / sweep rate Team 0/2 (%s): %.3f / %.3f / %.3f' %(p02_level,
        (sum(calls[0]['caller_points']==-2) + sum(calls[2]['caller_points']==-2)) / callcount_02,
        (sum((calls[0]['caller_points']==1)&(calls[0]['alone']==0)) + sum((calls[2]['caller_points']==1)&(calls[2]['alone']==0))) / callcount_02,
        (sum(calls[0]['caller_points']==2) + sum(calls[2]['caller_points']==2)) / callcount_02
    ))
    print('Euchre / single / sweep rate Team 1/3 (%s): %.3f / %.3f / %.3f' %(p13_level,
        (sum(calls[1]['caller_points']==-2) + sum(calls[3]['caller_points']==-2)) / callcount_13,
        (sum((calls[1]['caller_points']==1)&(calls[1]['alone']==0)) + sum((calls[3]['caller_points']==1)&(calls[3]['alone']==0))) / callcount_13,
        (sum(calls[1]['caller_points']==2) + sum(calls[3]['caller_points']==2)) / callcount_13
    ))
    
    # Statistic 4: Rate of loner successes (by team)
    loners = [c['alone']==1 for c in calls]
    print('Loner success rate Team 0/2 (%s): %.2f' %(p02_level, (sum(loners[0]['caller_points']==4)+sum(loners[2]['caller_points']==4))\
                                                                 /(len(loners[0])+len(loners[2]))))
    print('Loner success rate Team 0/2 (%s): %.2f' %(p02_level, (sum(loners[1]['caller_points']==4)+sum(loners[3]['caller_points']==4))\
                                                                 /(len(loners[1])+len(loners[3]))))

In [58]:
def get_performance_incremental(folder, p02_level, p13_level, prnt=True, every=20, max_count=None):
    totalcounts, callcounts, totallonercounts, successfullonercounts, euchrecounts, singlecounts, sweepcounts = 0,[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]
    totalpositivepoints, totalpoints = [0,0],[0,0]
    
    t = Timer(total=len(os.listdir(folder)), every=min(every, int(len(os.listdir(folder))/3)))
    for i, file in enumerate(os.listdir(folder)):
        # Look at the right files
        if '.csv' not in file: continue
        # Optionally stop early if dataset too large
        if max_count is not None and i > max_count: break
        # Optionally get ETA
        if prnt: t.get_eta(i)
        
        # Make the spacing look good for printing
        if len(p02_level) < len(p13_level): p02_level += ' '*(len(p13_level)-len(p02_level))
        else:                               p13_level += ' '*(len(p02_level)-len(p13_level))

        # Load df and get stats
        df = pd.read_csv(os.path.join(folder, file))
        df['caller_trueid'] = df.apply(lambda row: row['p' + str(row['caller']) + 'trueid'], axis=1)
        df['caller_points'] = 4*(df['result']=='Loner') + 2*(df['result']=='Sweep') + 1*(df['result']=='Single') - 2*(df['result']=='EUCHRE')
        calls = [df[df['caller_trueid']==i] for i in range(4)]
        loners = [c[c['alone']==1] for c in calls]
        masks = [(df['caller_trueid'] == 0) | (df['caller_trueid'] == 2), (df['caller_trueid'] == 1) | (df['caller_trueid'] == 3)]
        
        totalcounts += len(df)
        for j in range(2):
            callcounts[j] += len(calls[j])+len(calls[j+2])
            totallonercounts[j] += len(loners[j])+len(loners[j+2])
            successfullonercounts[j] += sum(loners[j]['caller_points']==4)+sum(loners[j+2]['caller_points']==4)
            euchrecounts[j] += sum(calls[j]['caller_points']==-2)+sum(calls[j+2]['caller_points']==-2)
            singlecounts[j] += sum((calls[j]['caller_points']==1) & (calls[j]['alone']==0))+\
                                sum((calls[j+2]['caller_points']==1) & (calls[j+2]['alone']==0))
            sweepcounts[j] += sum(calls[j]['caller_points']==2)+sum(calls[j+2]['caller_points']==2)
            
            totalpositivepoints[j] += sum(df[(masks[j]) & (df['caller_points']>0)]['caller_points'])
            totalpoints[j] += sum(calls[j]['caller_points'])+sum(calls[j+2]['caller_points'])
            
        
    # Statistic 1: Avg positive points (by team) per every hand
    print()
    print('Non-negative PPH Team 0/2 (%s): %.3f' %(p02_level, totalpositivepoints[0]/totalcounts))
    print('Non-negative PPH Team 1/3 (%s): %.3f' %(p13_level, totalpositivepoints[1]/totalcounts))
    print()

    # Statistic 2: Avg points (by team) per call
    print('PPC Team 0/2 (%s): %.3f' %(p02_level, totalpoints[0]/callcounts[0]))
    print('PPC Team 1/3 (%s): %.3f' %(p13_level, totalpoints[1]/callcounts[1]))
    print()

    # Statistic 3: Rate of euchres / singles / sweeps (by team) per call
    print('Euchre / single / sweep rate Team 0/2 (%s): %.3f / %.3f / %.3f' %(p02_level,
        euchrecounts[0]/callcounts[0], singlecounts[0]/callcounts[0], sweepcounts[0]/callcounts[0]))
    print('Euchre / single / sweep rate Team 1/3 (%s): %.3f / %.3f / %.3f' %(p13_level,
        euchrecounts[1]/callcounts[1], singlecounts[1]/callcounts[1], sweepcounts[1]/callcounts[1]))
    print()

    # Statistic 4: Rate of loner successes (by team)
    print('Loner success rate Team 0/2 (%s): %.2f' %(p02_level, successfullonercounts[0]/totallonercounts[0]))
    print('Loner success rate Team 1/3 (%s): %.2f' %(p13_level, successfullonercounts[1]/totallonercounts[1]))
    print()

In [68]:
get_performance_incremental(outfolder, 'optimal', 'basic')

ETA: ???, Now: 1:49:47 PM
Doing number 5 / 16  |  ETA: 1:49:50 PM, Now: 1:49:48 PM (2s remaining)
Doing number 10 / 16  |  ETA: 1:49:50 PM, Now: 1:49:49 PM (1s remaining)
Doing number 15 / 16  |  ETA: 1:49:50 PM, Now: 1:49:50 PM (0s remaining)

Non-negative PPH Team 0/2 (optimal): 0.504
Non-negative PPH Team 1/3 (basic  ): 0.504

PPC Team 0/2 (optimal): 0.596
PPC Team 1/3 (basic  ): 0.604

Euchre / single / sweep rate Team 0/2 (optimal): 0.205 / 0.609 / 0.147
Euchre / single / sweep rate Team 1/3 (basic  ): 0.203 / 0.608 / 0.150

Loner success rate Team 0/2 (optimal): 0.54
Loner success rate Team 1/3 (basic  ): 0.55



# Is there really no benefit to playing "optimally"???

In [212]:
p0_opt, p2_opt = opt.make_optimal_player(0), opt.make_optimal_player(2)
p1_opt, p3_opt = opt.make_optimal_player(1), opt.make_optimal_player(3)
p0_basic, p2_basic = opt.make_optimal_player(0), opt.make_optimal_player(2)
p1_basic, p3_basic = subopt.make_BASIC_player(1), subopt.make_BASIC_player(3)

board_opt = Board(p0=p0_opt, p1=p1_opt, p2=p2_opt, p3=p3_opt, seed=623)
board_basic = Board(p0=p0_basic, p1=p1_basic, p2=p2_basic, p3=p3_basic, seed=623)

In [213]:
%%time
seeds_optimal_better, seeds_no_difference, seeds_basic_better = [],[],[]
for i in range(10000):
    seed = np.random.randint(10000)
    
    board_opt.play_hand(seed=seed)
    board_basic.play_hand(seed=seed)
    
    net_team1_opt = board_opt.points_this_round[1]-board_opt.points_this_round[0]
    net_team1_basic = board_basic.points_this_round[1]-board_basic.points_this_round[0]
    if board_opt.data[-1] == board_basic.data[-1]:
        seeds_no_difference.append(seed)
    elif net_team1_opt == net_team1_basic:
        seeds_no_difference.append(seed)
    elif net_team1_opt > net_team1_basic:
        seeds_optimal_better.append(seed)
    elif net_team1_opt < net_team1_basic:
        seeds_basic_better.append(seed)
    else:
        print('Why am I here? Seed:', seed, 'Opt:', board_opt.points_this_round, 'Basic:', board_basic.points_this_round)

print('Count no diff:', len(seeds_no_difference))
print('Count optimal better:', len(seeds_optimal_better))
print('Count basic better:', len(seeds_basic_better))

Count no diff: 9491
Count optimal better: 213
Count basic better: 296
CPU times: user 51.2 s, sys: 86.1 ms, total: 51.3 s
Wall time: 51.2 s


# When basic is better, why?

In [266]:
seed = np.random.choice(seeds_basic_better)
print(seed)

board_opt.play_hand(seed=seed)
board_basic.play_hand(seed=seed)

hands_opt, globalstuff_opt, tricks_opt = Board.data_to_readable(board_opt.data[-1])
hands_basic, globalstuff_basic, tricks_basic = Board.data_to_readable(board_basic.data[-1])

print('Hands equal       :', hands_opt == hands_basic)
print('Global stuff equal:', globalstuff_opt == globalstuff_basic)

6781
Hands equal       : True
Global stuff equal: True


In [267]:
hands_opt

{3: ['9C', 'AS', 'JS', 'TS', '9S'],
 0: ['JH', 'TC', 'AH', '9H', 'KS'],
 1: ['KD', 'JC', 'KH', 'TH', 'QS'],
 2: ['QD', '9D', 'AC', 'KC', 'QH']}

In [268]:
globalstuff_opt # trumpsuit, callerid, dealerid, firstround, turncard

['S', 3, 2, False, 'TD']

In [269]:
tricks_opt

{0: (['JS', 'KS', 'QS', '9D'], 3),
 1: (['9S', '9H', 'JC', 'QD'], 1),
 2: (['KD', 'QH', 'TS', 'TC'], 3),
 3: (['AS', 'JH', 'TH', 'KC'], 3),
 4: (['9C', 'AH', 'KH', 'AC'], 2)}

In [270]:
tricks_basic

{0: (['JS', 'KS', 'QS', '9D'], 3),
 1: (['9C', 'TC', 'JC', 'KC'], 1),
 2: (['KD', 'QD', '9S', '9H'], 3),
 3: (['AS', 'JH', 'TH', 'QH'], 3),
 4: (['TS', 'AH', 'KH', 'AC'], 3)}

Seems like most of these cases are due to basic leading the highest trump when available, while optimal leads an off-ace. The off-ace gets trumped, which the highest trump would have sucked out.

# When optimal is better, why?

In [301]:
seed = np.random.choice(seeds_optimal_better)
print(seed)

board_opt.play_hand(seed=seed)
board_basic.play_hand(seed=seed)

hands_opt, globalstuff_opt, tricks_opt = Board.data_to_readable(board_opt.data[-1])
hands_basic, globalstuff_basic, tricks_basic = Board.data_to_readable(board_basic.data[-1])

print('Hands equal       :', hands_opt == hands_basic)
print('Global stuff equal:', globalstuff_opt == globalstuff_basic)

6205
Hands equal       : True
Global stuff equal: True


In [302]:
hands_opt

{3: ['JC', 'QC', 'TC', 'AH', 'TH'],
 0: ['QS', 'TS', '9S', 'AC', '9H'],
 1: ['AS', 'KC', '9C', 'KD', '9D'],
 2: ['QD', 'JD', 'KH', 'QH', 'JH']}

In [303]:
globalstuff_opt # trumpsuit, callerid, dealerid, firstround, turncard

['C', 3, 2, False, 'KS']

In [304]:
tricks_opt

{0: (['AH', '9H', 'AS', 'JH'], 3),
 1: (['JC', 'AC', '9C', 'JD'], 3),
 2: (['TH', '9S', 'KC', 'QH'], 1),
 3: (['KD', 'QD', 'TC', 'TS'], 3),
 4: (['QC', 'QS', '9D', 'KH'], 3)}

In [305]:
tricks_basic

{0: (['JC', 'AC', '9C', 'JD'], 3),
 1: (['AH', '9H', '9D', 'JH'], 3),
 2: (['TH', '9S', 'KD', 'KH'], 2),
 3: (['QH', 'TC', 'TS', 'AS'], 3),
 4: (['QC', 'QS', 'KC', 'QD'], 1)}

# Extra

# Create optimal-vs-intermediate dataset

In [16]:
if os.path.exists('/staging/'):
    outfolder = '/staging/fast/taylora/euchre/opt_vs_inter'
else:
    outfolder = 'opt_vs_inter'
if not os.path.exists(outfolder): os.mkdir(outfolder)
p0, p2 = opt.make_optimal_player(0), opt.make_optimal_player(2)
p1, p3 = subopt.make_INTERMEDIATE_player(1), subopt.make_INTERMEDIATE_player(3)

In [17]:
n_epochs, n_hands = int(1e3), int(1e5)
AVAILABLE_CPUS = 16-1
particular_performance_function = partial(performance, n_epochs=int(n_epochs/AVAILABLE_CPUS), n_hands=n_hands, p0=p0, p1=p1, p2=p2, p3=p3)

In [21]:
%%time
z="""pool = multiprocessing.Pool(AVAILABLE_CPUS)
for worker in range(AVAILABLE_CPUS):
    pool.apply_async(particular_performance_function, args=(worker,))
pool.close()
pool.join()
print('Done!')"""

CPU times: user 0 ns, sys: 2 µs, total: 2 µs
Wall time: 4.05 µs


In [18]:
!ls /staging/fast/taylora/euchre

RLdataset  old_thresholds  opt_vs_basic  opt_vs_inter  tfrecords


In [4]:
!ls /staging/fast/taylora/euchre/opt_vs_inter

100000_hands_000.csv  100000_hands_330.csv  100000_hands_660.csv
100000_hands_001.csv  100000_hands_331.csv  100000_hands_661.csv
100000_hands_002.csv  100000_hands_332.csv  100000_hands_662.csv
100000_hands_003.csv  100000_hands_333.csv  100000_hands_663.csv
100000_hands_004.csv  100000_hands_334.csv  100000_hands_664.csv
100000_hands_005.csv  100000_hands_335.csv  100000_hands_665.csv
100000_hands_006.csv  100000_hands_336.csv  100000_hands_666.csv
100000_hands_007.csv  100000_hands_337.csv  100000_hands_667.csv
100000_hands_008.csv  100000_hands_338.csv  100000_hands_668.csv
100000_hands_009.csv  100000_hands_339.csv  100000_hands_669.csv
100000_hands_010.csv  100000_hands_340.csv  100000_hands_670.csv
100000_hands_011.csv  100000_hands_341.csv  100000_hands_671.csv
100000_hands_012.csv  100000_hands_342.csv  100000_hands_672.csv
100000_hands_013.csv  100000_hands_343.csv  100000_hands_673.csv
100000_hands_014.csv  100000_hands_344.csv  100000_hands_674.csv
100000_hands_015.csv  100

# Create intermediate-vs-basic dataset

In [17]:
if os.path.exists('/staging/'):
    outfolder = '/staging/fast/taylora/euchre/inter_vs_basic'
else:
    outfolder = 'inter_vs_basic'
if not os.path.exists(outfolder): os.mkdir(outfolder)
p0, p2 = subopt.make_BASIC_player(0), subopt.make_BASIC_player(2)
p1, p3 = subopt.make_INTERMEDIATE_player(1), subopt.make_INTERMEDIATE_player(3)

In [18]:
n_epochs, n_hands = int(1e3), int(1e5)
AVAILABLE_CPUS = 16-1
particular_performance_function = partial(performance, n_epochs=int(n_epochs/AVAILABLE_CPUS), n_hands=n_hands, p0=p0, p1=p1, p2=p2, p3=p3)

In [20]:
%%time
z="""
pool = multiprocessing.Pool(AVAILABLE_CPUS)
for worker in range(AVAILABLE_CPUS):
    pool.apply_async(particular_performance_function, args=(worker,))
pool.close()
pool.join()
print('Done!')"""

CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 79.9 µs


In [8]:
!ls /staging/fast/taylora/euchre/inter_vs_basic